In [1]:
# List all device
from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [2]:
# Check available GPU
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";  

In [4]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
import pickle

In [5]:
with open("../data/dowjones_calculated/periods.txt", "rb") as fp:   # Unpickling
    dataset = pickle.load(fp)

In [6]:
X_train = dataset[0][0][0].values
y_train = dataset[0][0][1].values
X_test = dataset[1][0][0].values
y_test = dataset[1][0][1].values

In [7]:
y_train.shape

(750, 31)

In [8]:
X_test.shape

(250, 31)

In [9]:
y_test.shape

(250, 31)

In [10]:
# Reshaping X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [12]:
# The LSTM architecture
regressor = Sequential()
# First LSTM layer with Dropout regularisation
regressor.add(LSTM(units=25, return_sequences=True, input_shape=(X_train.shape[1],1)))
regressor.add(Dropout(0.1))
# Second LSTM layer
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))
# The output layer
regressor.add(Dense(31, activation='softmax'))
# Compiling the RNN
regressor.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Fitting to the training set
regressor.fit(X_train,y_train,epochs=5,batch_size=32)

Epoch 1/5
750/750 [==============================] - 2s 3ms/step - loss: 1.7879 - binary_crossentropy: 1.7879
Epoch 2/5
750/750 [==============================] - 1s 2ms/step - loss: 1.7868 - binary_crossentropy: 1.7868
Epoch 3/5
750/750 [==============================] - 1s 2ms/step - loss: 1.7857 - binary_crossentropy: 1.7857
Epoch 4/5
750/750 [==============================] - 1s 2ms/step - loss: 1.7850 - binary_crossentropy: 1.7850
Epoch 5/5
750/750 [==============================] - 1s 2ms/step - loss: 1.7826 - binary_crossentropy: 1.7826


In [ ]:
# Evaluating our model
predicted = regressor.predict(X_test)

In [ ]:
y_test.shape

In [ ]:
predicted.shape

In [ ]:
y_test[0]

In [ ]:
predicted[0]

In [ ]:
predicted[:3]

In [ ]:
label = predicted > 0.5
label = label * 1 # Convert boolean to int
label

In [ ]:
(sum(y_test == label)/len(y_test)).mean()

In [ ]:
# from keras.models import load_model

# regressor.save('../model/LSTM/my_model.h5')  # creates a HDF5 file 'my_model.h5'
# del regressor  # deletes the existing model

# # returns a compiled model
# # identical to the previous one
# regressor1 = load_model('../model/LSTM/my_model.h5')
# regressor1.predict(X_test)